In [ ]:
################################################################################################
### This code uses scripts adapted from RivWidthCloud for the GEE Python API to pull river surface temperatures ###
#############################################################################################################
### GOAL: Pull in the dams and areas of interest, get list of Landsat images, get dynamic centerlines, extract temperatures to a csv ###
#############################################################################################################

In [ ]:
# Import the packages needed
import ee
import geemap
import pandas as pd
import time
from collections import defaultdict
ee.Initialize()

In [ ]:
#####################
### Prepare the data ###
#####################

In [ ]:
##### Pull in Dam Locations in from GEE assets ########
GrodDams = ee.FeatureCollection('Insert_GEE_Asset_Name_here') 

In [ ]:
### Starting with SWORD River Nodes ###
# Import the files for the river nodes from GEE assets
rivernodes = ee.FeatureCollection('Insert_GEE_Asset_Name_here') ## Use Cleaned Version of the output from the R Script

# Create a function to buffer all the river nodes and apply the function
def bufferFun(f):
  bufferPoint = f.buffer(30)
  return(bufferPoint)

# Apply buffer function
rivernodes_buffer = rivernodes.map(bufferFun)

In [ ]:
### Dam Names ###
# Get list of distinct dams for selecting and extracting for export
# This will make sure that we are getting the same images as the temp extraction code
dams = rivernodes.aggregate_array('Assgn_dam').distinct().getInfo()
dams.sort()
print('List of distinct dams:', dams)

In [ ]:
# Filter into Parts for processing
dams = dams[:] #Update this as needed for a given run

In [ ]:
### Set Imagery Parameters ###
# Add Earth Engine dataset
TempProduct = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')

# Set parameters for the images, update this as needed for a given run
date_start = ee.Date('2013-03-01')
date_end = ee.Date('2024-12-31')

In [ ]:
###############################################
### Loop Thru Dams for Centerlines & Temperatures ###
###############################################

In [ ]:
# Create a For Loop to go through list of the dams, select the points, run the RWC
for i in range(len(dams)):
    start_tot = time.time()
    #Filter the dam locations
    Dam_Point_filt = GrodDams.filter(ee.Filter.eq('grod_id', dams[i]))    
    # Filter buffer to dam
    Buffer_filt = rivernodes_buffer.filter(ee.Filter.eq('Assgn_dam', dams[i]))                        
    # Filter the AOI to dam 
    Nodes_filt = rivernodes.filter(ee.Filter.eq('Assgn_dam', dams[i]))
    # Get the bounds of the points
    AOI = Nodes_filt.geometry().bounds()
    
    ## GET THE IMAGE LIST ##
    # Filter the Image Collection with the defined parameters
    LandsatTempProd = TempProduct.filterBounds(AOI).filterDate(date_start, date_end).filter(ee.Filter.lte('CLOUD_COVER', 10)).filterBounds(Dam_Point_filt)
    # Make a list of the image ids needed for RivWidthCloud
    LoopList = LandsatTempProd.aggregate_array('LANDSAT_PRODUCT_ID').distinct().getInfo()
    # Convert to a datafram and append
    LoopList_df = pd.DataFrame(LoopList)
    
    # Prep for Export to CSV (format for batch RivWidthCloud export)
    LoopList_df.rename(columns={0:'LANDSAT_ID'}, inplace=True ) # naming the column
    # save image to CSV file
    LoopList_df.to_csv('River_Batch/Landsat_Ids_Dam'+ str(dams[i]) + '_extra' + '.csv', index=False) # One to keep

    ## RUN RIVWIDTHCLOUD BATCH ##
    for j in range(len(LoopList)):
        DamNo = dams[i]
        Filename = 'Dam '+ str(dams[i])
        LandsatID = LoopList[j]
        start = time.time()
        %run RWC_Landsat_Temps.py $LandsatID -p -d 100000 -dam $DamNo
        end = time.time()
        print('Run Time: '+ str(((end-start) * 10**3)/60000) + 'min')
    end_tot = time.time()
    print('Dam '+ str(dams[i])+ ': RWC and Temp Complete \n Total Run Time: '+ str(((end_tot-start_tot)* 10**3)/60000)+ 'min')

    ### Note: Have to change export location in source code  -- Py file: RWC_Landsat_Temps , line 151-- ###
    ### Note: Have to change export location in source code to update output message -- Py file: FC_to_Desktop , line 51-- ###